Import the standard libraries to work with dataframes

In [1]:
import numpy as np
import pandas as pd

Use the read_html function of from the Pandas library and pass the url of the wikipedia page. This returns a list of dataframes. These dataframes are assigned to the variable 'dfs'

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

Display the first five dfs

In [3]:
dfs[0:5]

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

Assign the first dataframe from dfs to the dataframe: df_toronto

In [4]:
df_toronto = dfs[0]

Display first 8 rows in this dataframe

In [5]:
df_toronto.head(8)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned


Created a copy of this dataframe for data wrangling

In [6]:
df_toronto_w = df_toronto.copy()

Iterate through each row in dataframe and if the borough for that row is listed as 'Not assigned', removed the row from the dataframe

In [7]:
for index, row in df_toronto_w.iterrows():
    if row['Borough'] == 'Not assigned':
        df_toronto_w.drop(index, inplace=True)

Search for any rows where the Neighbourhood is listed as 'Not Assigned'. Result returns no records.

In [8]:
df_toronto_w.loc[df_toronto_w['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


Dispaly resulting dataframe. This will show the indices having been removed. Will then reset index in the following cell.

In [9]:
df_toronto_w.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df_toronto_w.reset_index(drop=True, inplace=True)
df_toronto_w.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Using the shape function, indicate the number of rows in resulting dataframe.

In [11]:
print('There are ', df_toronto_w.shape[0], ' rows in this dataframe')

There are  103  rows in this dataframe


Reading the postal codes CSV from site below to obtain latitude and longitude coordinates of each postal code abve.

In [12]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')

Place the coordinates found above into the dataframe previously created using merge.

In [13]:
df_toronto_w = df_toronto_w.merge(coords, left_on='Postal Code', right_on='Postal Code')

Use the groupby and count functions to review number of neighbourhoods in each borough.

In [14]:
df_toronto_w.groupby('Borough').count()

,Postal Code,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


Based on results above, will be reviewing neighbourhoods within the North York borough. Using geopy, obtain the coordinates of North York.

In [15]:
from geopy.geocoders import Nominatim

In [16]:
address = 'North York Ontario'

geolocator = Nominatim(user_agent="nyork_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York borough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York borough are 43.7543263, -79.44911696639593.


Installing and importing folium to create maps.

In [17]:
!pip install folium==0.5.0 # uncomment this line if you haven't completed the Foursquare API lab

     |████████████████████████████████| 79 kB 7.0 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=a714a035b098eb2f392a33ba0beb323ea68f61dab5f1772f750dd0518b452bbf
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [18]:
import folium

In [19]:
# create map of New York using latitude and longitude values
map_nyork = folium.Map(location=[latitude, longitude], zoom_start=12)

In [20]:
map_nyork

Create dataframe to contain only North York data.

In [21]:
nyork_data = df_toronto_w[df_toronto_w['Borough'] == 'North York'].reset_index(drop=True)

Using the new dataframe, add markers to map focused on North York

In [22]:
for lat, lng, label in zip(nyork_data['Latitude'], nyork_data['Longitude'], nyork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyork)  
    
map_nyork

Enter Foursquare API credentials

In [23]:
# The code was removed by Watson Studio for sharing.

In [24]:
nyork_data.loc[0, 'Neighbourhood']

'Parkwoods'

In [25]:
neighbourhood_latitude = nyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = nyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = nyork_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

Import libraries below to work with json files from Foursquare

In [27]:
import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Function getNearbyVenues as shown in lab.

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Using the above function, get venues from the North York specific dataframe.

In [29]:
nyork_venues = getNearbyVenues(names=nyork_data['Neighbourhood'],
                                   latitudes=nyork_data['Latitude'],
                                   longitudes=nyork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


Group the venues by Neighbourhood

In [30]:
nyork_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Don Mills,26,26,26,26,26,26
Downsview,14,14,14,14,14,14
"Fairview, Henry Farm, Oriole",69,69,69,69,69,69
Glencairn,6,6,6,6,6,6
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2


Analyze each neighbourhood

In [31]:
# one hot encoding
nyork_onehot = pd.get_dummies(nyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
nyork_onehot['Neighbourhood'] = nyork_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [nyork_onehot.columns[-1]] + list(nyork_onehot.columns[:-1])
nyork_onehot = nyork_onehot[fixed_columns]

nyork_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


####  Group rows by neighbourhood by taking the mean of the frequency of occurrence of each category of venues

In [32]:
nyork_grouped = nyork_onehot.groupby('Neighbourhood').mean().reset_index()
nyork_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,...,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.038462,0.000000,0.038462,0.038462,0.000000,0.000000,...,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.014493,0.000000,0.000000,0.014493,0.000000,0.028986,0.028986,...,0.000000,0.014493,0.000000,0.000000,0.014493,0.028986,0.0,0.014493,0.000000,0.014493
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.166667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


Find the top venue categories in each neighbourhood

In [33]:
num_top_venues = 5

for hood in nyork_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = nyork_grouped[nyork_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                   Pharmacy  0.05
3              Shopping Mall  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4               Lounge  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3         Pizza Place  0.04
4       Grocery Store  0.04


----Don Mills----
                 venue  freq
0                  Gym  0.12
1          Coffee Shop  0.08
2           Restaurant  0.08
3  Japanese Restaurant  0.08
4           Beer Store  0.08


----Downsview----
            venue  freq
0   Grocery Store  0.21
1            Park  0.14
2    Liquor Store  0.07
3      Food 

Function from lab to find most common venue types

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = nyork_grouped['Neighbourhood']

for ind in np.arange(nyork_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyork_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Bridal Shop,Ice Cream Shop,Health Food Store,Mobile Phone Shop,Restaurant,Deli / Bodega,Sandwich Place
1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Food Court,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Greek Restaurant,Breakfast Spot,Comfort Food Restaurant,Hobby Shop,Pub,Liquor Store
3,Don Mills,Gym,Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Dim Sum Restaurant,Discount Store,Bubble Tea Shop,Asian Restaurant,Bike Shop
4,Downsview,Grocery Store,Park,Bank,Business Service,Airport,Food Truck,Discount Store,Athletics & Sports,Liquor Store,Baseball Field


Import KMeans to preform clustering algorithms

In [36]:
from sklearn.cluster import KMeans

In [37]:
# set number of clusters
kclusters = 5

nyork_grouped_clustering = nyork_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 2], dtype=int32)

In [38]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nyork_merged = nyork_data

# merge nyork_grouped with nyork_data to add latitude/longitude for each neighborhood
nyork_merged = nyork_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

nyork_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Construction & Landscaping,Convenience Store,Department Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Women's Store,Deli / Bodega,Chinese Restaurant,Chocolate Shop,Clothing Store,Comfort Food Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Gift Shop,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Vietnamese Restaurant,Accessories Store,Supermarket
3,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Dim Sum Restaurant,Discount Store,Bubble Tea Shop,Asian Restaurant,Bike Shop
4,M6B,North York,Glencairn,43.709577,-79.445073,1,Park,Asian Restaurant,Pizza Place,Bakery,Pub,Japanese Restaurant,Women's Store,Deli / Bodega,Chocolate Shop,Clothing Store


A number of fields in above dataframe contain Nan values so used the function 'dropna' to remove these to ensure the map markers can be placed in following cells. Also imported necessary classes from matplotlip to assist with map created. Finally, the map of North York with the computed clusters is displayed.

In [39]:
nyork_merged = nyork_merged.dropna()

In [40]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyork_merged['Latitude'], nyork_merged['Longitude'], nyork_merged['Neighbourhood'], nyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters